## Select how many hidden layer and neurons in a neural network

In [1]:

# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd



In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from keras.models import Sequential
from keras.layers import Dense, Activation, Embedding, Flatten, LeakyReLU, BatchNormalization, Dropout
from keras.activations import relu, sigmoid
from keras.layers import LeakyReLU

from sklearn.preprocessing import StandardScaler

Using TensorFlow backend.


In [3]:
# Importing the dataset
dataset = pd.read_csv('Telco-Customer-Churn.csv')
# X = dataset.iloc[:, 1:19].values
# y = dataset.iloc[:, 20].values



In [4]:
dataset["churn"]=dataset.Churn.map(dict(Yes=1, No=0))
dataset.drop(['Churn'],axis=1,inplace=True)


In [5]:
dataset.shape

(7043, 21)

In [5]:
dataset.drop(['PaymentMethod'],axis=1,inplace=True)

In [43]:
# geography=pd.get_dummies(dataset["Geography"],drop_first=True)
# gender=pd.get_dummies(dataset['Gender'],drop_first=True)

In [6]:
#categorical data
categorical_cols = ['Partner', 'Dependents', 'PhoneService', 'InternetService','OnlineSecurity','OnlineBackup','DeviceProtection','TechSupport','StreamingTV',
                   'StreamingMovies','PaperlessBilling'] 

#import pandas as pd
df = pd.get_dummies(dataset, columns = categorical_cols)

In [62]:
def con(val):
    """
    Convert the string number value to a float
     - Remove $
     - Remove commas
     - Convert to float type
    """
    new_val = val.replace(',','')
    return float(new_val)

In [7]:
df['totalcharges']=pd.to_numeric(df['TotalCharges'], errors='coerce')


In [8]:
df.drop(['TotalCharges'],axis=1,inplace= True)

In [9]:
df.drop(['customerID'],axis=1,inplace= True)
df.drop(['gender'],axis=1,inplace= True)
df.drop(['MultipleLines'],axis=1,inplace= True)
df.drop(['Contract'],axis=1,inplace= True)


In [10]:
df.dtypes

SeniorCitizen                             int64
tenure                                    int64
MonthlyCharges                          float64
churn                                     int64
Partner_No                                uint8
Partner_Yes                               uint8
Dependents_No                             uint8
Dependents_Yes                            uint8
PhoneService_No                           uint8
PhoneService_Yes                          uint8
InternetService_DSL                       uint8
InternetService_Fiber optic               uint8
InternetService_No                        uint8
OnlineSecurity_No                         uint8
OnlineSecurity_No internet service        uint8
OnlineSecurity_Yes                        uint8
OnlineBackup_No                           uint8
OnlineBackup_No internet service          uint8
OnlineBackup_Yes                          uint8
DeviceProtection_No                       uint8
DeviceProtection_No internet service    

In [29]:
X = df.iloc[:, 1:34].values
y = df.iloc[:,:34].values

In [25]:
df.dtypes.index.to_list()

['SeniorCitizen',
 'tenure',
 'MonthlyCharges',
 'Partner_No',
 'Partner_Yes',
 'Dependents_No',
 'Dependents_Yes',
 'PhoneService_No',
 'PhoneService_Yes',
 'InternetService_DSL',
 'InternetService_Fiber optic',
 'InternetService_No',
 'OnlineSecurity_No',
 'OnlineSecurity_No internet service',
 'OnlineSecurity_Yes',
 'OnlineBackup_No',
 'OnlineBackup_No internet service',
 'OnlineBackup_Yes',
 'DeviceProtection_No',
 'DeviceProtection_No internet service',
 'DeviceProtection_Yes',
 'TechSupport_No',
 'TechSupport_No internet service',
 'TechSupport_Yes',
 'StreamingTV_No',
 'StreamingTV_No internet service',
 'StreamingTV_Yes',
 'StreamingMovies_No',
 'StreamingMovies_No internet service',
 'StreamingMovies_Yes',
 'PaperlessBilling_No',
 'PaperlessBilling_Yes',
 'totalcharges',
 'churn']

In [12]:
df = df[['SeniorCitizen',
 'tenure',
 'MonthlyCharges',
 'Partner_No',
 'Partner_Yes',
 'Dependents_No',
 'Dependents_Yes',
 'PhoneService_No',
 'PhoneService_Yes',
 'InternetService_DSL',
 'InternetService_Fiber optic',
 'InternetService_No',
 'OnlineSecurity_No',
 'OnlineSecurity_No internet service',
 'OnlineSecurity_Yes',
 'OnlineBackup_No',
 'OnlineBackup_No internet service',
 'OnlineBackup_Yes',
 'DeviceProtection_No',
 'DeviceProtection_No internet service',
 'DeviceProtection_Yes',
 'TechSupport_No',
 'TechSupport_No internet service',
 'TechSupport_Yes',
 'StreamingTV_No',
 'StreamingTV_No internet service',
 'StreamingTV_Yes',
 'StreamingMovies_No',
 'StreamingMovies_No internet service',
 'StreamingMovies_Yes',
 'PaperlessBilling_No',
 'PaperlessBilling_Yes',
 'totalcharges',
 'churn']]


### Modeling

In [48]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

# Feature Scaling
# from sklearn.preprocessing import StandardScaler
# sc = StandardScaler()
# X_train = sc.fit_transform(X_train)
# X_test = sc.transform(X_test)


In [49]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

In [50]:
def create_model(layers, activation):
    model = Sequential()
    for i, nodes in enumerate(layers):
        if i==0:
            model.add(Dense(nodes,input_dim=X_train.shape[1]))
            model.add(Activation(activation))
        else:
            model.add(Dense(nodes))
            model.add(Activation(activation))
    model.add(Dense(1)) # Note: no activation beyond this point
    
    model.compile(optimizer='adam', loss='binary_crossentropy',metrics=['accuracy'])
    return model
    
model = KerasClassifier(build_fn=create_model, verbose=0)

In [51]:
model

In [52]:
layers = [(20,), (40, 20), (45, 30, 15)]
#layers=[(150), (160,100), (140,100,500)]
activations = ['sigmoid', 'relu']
param_grid = dict(layers=layers, activation=activations, batch_size = [128, 256], epochs=[30])
grid = GridSearchCV(estimator=model, param_grid=param_grid)

In [55]:
grid_result = grid.fit(X_train, y_train)

In [56]:
[grid_result.best_score_,grid_result.best_params_]

In [21]:
pred_y = grid.predict(X_test)

In [22]:
y_pred = (pred_y > 0.5)

In [23]:
y_pred

array([[False],
       [False],
       [False],
       ...,
       [False],
       [False],
       [False]])

In [24]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)

In [25]:
cm

array([[1595,    0],
       [ 405,    0]], dtype=int64)

In [26]:
from sklearn.metrics import accuracy_score
score=accuracy_score(y_test,y_pred)

In [27]:
score

0.7975